# 1 Import Library

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
from unicodedata import normalize
from wordcloud import WordCloud
from tqdm.notebook import tqdm
from collections import Counter
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import unicodedata
import time
import re
import sys
import os
import platform
import warnings

In [ ]:
# 한글 깨짐 방지
path = "c:/Windows/Fonts/NANUMBARUNGOTHIC.ttf"
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')    

plt.rcParams['axes.unicode_minus'] = False

# 2 Crawling

## 2.1 Instagram Crawling

### 2.1.1 함수 정의

In [9]:
def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/' + word
    return url

In [11]:
def select_first(driver):
    first = driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(3)

select_first(driver)

In [ ]:
def move_next(driver):
    right = driver.find_element_by_css_selector(
        'a._65Bje.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

In [1]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 1. 본문내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
        content = unicodedata.normalize('NFC', content)

    except:
        content = ' ' 

    # 2. 본문 내용 해시태그(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)

    # 3. 작성일시
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]

    # 4. 좋아요수
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]
    except:
        like = 0

    # 5. 위치정보
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)

    except:
        place = ''

    # 6. 수집한 정보 저장하기
    data = [content, tags, date, like, place]
    return data

### 2.1.2 word "프레시지"

In [2]:
driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get('https://www.instagram.com')
time.sleep(2)

In [8]:
email = '********'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '********'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)

input_pw.submit()
time.sleep(3)

In [10]:
word = '프레시지'
url = insta_searching(word)
driver.get(url)

In [19]:
# 인스타그램 검색 페이지 URL 만들기
word = '프레시지'
url = insta_searching(word)

driver.get(url)
time.sleep(3)

# 첫 번째 게시글 열기
select_first(driver)

# 비어 있는 변수(result) 만들기
result = []

# 여러 게시물 수집하기
target = 1000

for i in tqdm(range(target)):
    try:
        data = get_content(driver)
        result.append(data)
        move_next(driver)

    except:
        time.sleep(2)
        move_next(driver)

print(result[:2])


[['#금집네-브로콜리세척하기귀찮아계속냉장고에방치해놨더니예쁜프리지아될뻔🌼먹어도갠찮겠지,,🙇🏻\u200d♀️--#눈뜨자마자빨간맛#프레시지#짬뽕순두부#계란장조림#고추부각', ['#금집네-브로콜리세척하기귀찮아계속냉장고에방치해놨더니예쁜프리지아될뻔🌼먹어도갠찮겠지', '#눈뜨자마자빨간맛', '#프레시지', '#짬뽕순두부', '#계란장조림', '#고추부각'], '2020-12-19', '1,185', '우리집'], ['#신짱키친🎁홈파티 메리 투 스테이크 세트“(42,600원👉🏻27,900원)= 35%할인중”.🌐”YouTube” 신짱키친[SJJANG KitChen]에 오시면 더 많은 영상을 보실수 있습니다...🎉세트구성- 피에스타 스테이크 2개- 쉬림프 로제파스타 1개- 리코타치즈 샐러드 1개🚀무료배송입니다..맛있으니깐 믿고 드세유!!홈파티 메리 투 스테이크 세트는2명에서 먹기 딱 좋아요👍🏻(링크 클릭이 안되시는분들은)“인스타를 끄고 다시 켜보시면 됩니다”안스타의 오류에유 그게ㅠㅠㅠㅠㅠ이번 크리스마스는 메리 투 스테이크세트로 배부르게 보내봅시다🐷🐷.✅”메리 투 스테이크 세트” 구매는 프로필 링크 Click!..#홈파티음식 #소고기스테이크 #쉬림프로제파스타 #스테이크세트 #프레시지 #윙잇 #쿠킹박스 #밀키트 #크리스마스이벤트 #크리스마스할인 #크리스마스파티 #연말파티 #요리영상 #요리촬영 #마미스팟그릴 #간편요리 #간단한요리 #steaks #마미스팟', ['#신짱키친🎁홈파티', '#홈파티음식', '#소고기스테이크', '#쉬림프로제파스타', '#스테이크세트', '#프레시지', '#윙잇', '#쿠킹박스', '#밀키트', '#크리스마스이벤트', '#크리스마스할인', '#크리스마스파티', '#연말파티', '#요리영상', '#요리촬영', '#마미스팟그릴', '#간편요리', '#간단한요리', '#steaks', '#마미스팟'], '2019-12-16', '3,353', '']]


In [20]:
len(result)

979

In [38]:
result_df = pd.DataFrame(result)
result_df.columns = ['content', 'tags', 'date', 'like', 'place']
result_df.to_excel('../output/fresheasy.xlsx')
driver.close()

### 2.1.3 word "프레시지밀키트"

In [34]:
driver = webdriver.Chrome('../driver/chromedriver.exe')

driver.get('https://www.instagram.com')
time.sleep(2)

email = '********'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '********'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)

input_pw.submit()
time.sleep(3)

In [30]:
word = '프레시지밀키트'
url = insta_searching(word)

driver.get(url)
time.sleep(3)

# 첫 번째 게시글 열기
select_first(driver)

# 비어 있는 변수(result) 만들기
result = []

# 여러 게시물 수집하기
target = 500

for i in tqdm(range(target)):
    try:
        data = get_content(driver)
        result.append(data)
        move_next(driver)

    except:
        time.sleep(2)
        move_next(driver)

print(result[:2])


[['#저녁밥상 #저녁밥상메뉴 #함박스테이크추천⠀쿠팡에서 판매하는 함박이너무 리뷰가 좋아서한번 구매해봤어요야채까지 있는 밀키트라서너무좋더라구요!!더좋은건 보통 냉동 함박은ㅠ납작하고 햄같은 느낌인데요건 딱 수제 느낌ㄷㄷ너무 맛있게 한끼 잘먹었어윱😍함박 밀키트 ➡️ 쿠팡 프레시지#밥상스타그램 #밥집추천 #집밥스타그램 #집밥🍚 #함박스텍 #함박스테이크밀키트 #계란후라이반숙 #밀키트추천#프레시지밀키트#먹팔 #먹스타소통 #경양식스타일', ['#저녁밥상', '#저녁밥상메뉴', '#함박스테이크추천⠀쿠팡에서', '#밥상스타그램', '#밥집추천', '#집밥스타그램', '#집밥🍚', '#함박스텍', '#함박스테이크밀키트', '#계란후라이반숙', '#밀키트추천', '#프레시지밀키트', '#먹팔', '#먹스타소통', '#경양식스타일'], '2020-10-05', '1,187', ''], ['요즘 밀키트제품 잘나와서아들 좋아하는메뉴로 사놓으면좋은애미 코스프레용으로 최고😍스테이크세트 해주면 나 한조각주고혼자 토마토까지 전부 클리어ㅋㅋㅋ그런데 이게 왜 2인세트이쥐?😅😅....#원주일상#일상그램#일상스타그램 #고기#고기스타그램#스테이크#홈쿡#프레시지#프레시지밀키트#밀키트#프레시지블랙라벨스테이크#소고기#더반찬#아들간식#식사#food#foodstagram#plating#빌보#빌레로이앤보흐#빌보뉴웨이브 #먹방', ['#원주일상', '#일상그램', '#일상스타그램', '#고기', '#고기스타그램', '#스테이크', '#홈쿡', '#프레시지', '#프레시지밀키트', '#밀키트', '#프레시지블랙라벨스테이크', '#소고기', '#더반찬', '#아들간식', '#식사', '#food', '#foodstagram', '#plating', '#빌보', '#빌레로이앤보흐', '#빌보뉴웨이브', '#먹방'], '2020-12-18', '840', '']]


In [31]:
len(result)

491

In [32]:
result_df = pd.DataFrame(result)
result_df.columns = ['content', 'tags', 'date', 'like', 'place']
result_df.to_excel('../output/fresheasymealkit_1.xlsx')

driver.close()

#### word "프레시지 밀키트" 추가 크롤링

In [38]:
word = '프레시지밀키트'
url = insta_searching(word)

driver.get(url)
time.sleep(3)

# 비어 있는 변수(result) 만들기
result = []

# 여러 게시물 수집하기
target = 145

for i in tqdm(range(target)):
    try:
        data = get_content(driver)
        result.append(data)
        move_next(driver)

    except:
        time.sleep(2)
        move_next(driver)

print(result[:2])


[['#오늘의점심 #홈런치 #트러플크림파스타 #홈브런치 #프레시지 #프레시지밀키트 #파스타 #홈메이드 #플레이팅', ['#오늘의점심', '#홈런치', '#트러플크림파스타', '#홈브런치', '#프레시지', '#프레시지밀키트', '#파스타', '#홈메이드', '#플레이팅'], '2019-12-10', '20', ''], ['#그저께일기.슬슬이가 해준 밀푀유나베넘 맛있어서 비법을 전수 받았다.나도 사.. 해먹어봐야지 #프레시지밀키트어디에서 어떻게 구입해야 GOOD!! Ok.. ...노는 내내 내이름 제대로 안부르고다른이모 찾길래 내 이름 못외우나싶었더니 가려고 인사하니 신나게 "윤미이모 잘가>ㅅ<" 열번도 더 외치는#밀당남박리우 쪼꼬미 이제 이모가 질리게 놀러갈꺼야..💁\u200d♀️#갓조이모', ['#그저께일기.슬슬이가', '#프레시지밀키트어디에서', '#밀당남박리우', '#갓조이모'], '2019-12-05', '21', '']]


In [41]:
result_df = pd.DataFrame(result)
result_df.columns = ['content', 'tags', 'date', 'like', 'place']
result_df.to_excel('../output/fresheasymealkit_2.xlsx', index=False)

driver.close()

In [61]:
fresheasymealkit1 = pd.read_excel('../output/fresheasymealkit_1.xlsx', engine='openpyxl')
fresheasymealkit2 = pd.read_excel('../output/fresheasymealkit_2.xlsx', engine='openpyxl')
fresheasymealkit = pd.concat([fresheasymealkit1, fresheasymealkit2])
fresheasymealkit.reset_index(inplace = True, drop=True)

In [62]:
len(fresheasymealkit)

636

In [68]:
fresheasymealkit

,content,tags,date,like,place
0,#저녁밥상 #저녁밥상메뉴 #함박스테이크추천⠀쿠팡에서 판매하는 함박이너무 리뷰가 좋아...,"['#저녁밥상', '#저녁밥상메뉴', '#함박스테이크추천⠀쿠팡에서', '#밥상스타그...",2020-10-05,"1,187",NaN
1,요즘 밀키트제품 잘나와서아들 좋아하는메뉴로 사놓으면좋은애미 코스프레용으로 최고😍스테...,"['#원주일상', '#일상그램', '#일상스타그램', '#고기', '#고기스타그램'...",2020-12-18,840,NaN
2,"매운거,자극적인거 너무 좋아해서남편의 잔소리를 피할수없지만 포기할수 없쥐~~~😤.....","['#프레시지', '#프레시지밀키트', '#밀키트', '#신혼밥상', '#신혼', ...",2019-10-23,455,NaN
3,예전에 놀러갔을때 해먹어본 프레시지 감바스 밀키트😋 오일도 넉넉하니 진짜 맛있었당❤...,"['#프레시지', '#프레시지밀키트', '#프레시지감바스', '#감바스밀키트', '...",2020-08-31,119,NaN
4,고기사랑 ♡편리한 스테이크 밀키트 먹방 😊#스테이크 #스테이크밀키트 #집밥 #고기 ...,"['#스테이크', '#스테이크밀키트', '#집밥', '#고기', '#고기사랑', '...",2020-12-01,406,NaN
...,...,...,...,...,...
631,🍳스테이크전문점 아니고홈쿡데스!옴마쉐프 요리부심 뿜뿜해도 되겠져?.🕞15분만에 이거...,"['#프레시지밀키트', '#프레시지', '#쁘띠텐더스테이크', '#그랜드비프스테이크...",2018-08-23,"1,056",NaN
632,늦은 #점심.간단하지만 푸짐하게 먹고 싶어#자이언트미트볼파스타 꺼내욤.빠르게 배송반...,"['#점심.간단하지만', '#자이언트미트볼파스타', '#밀키트', '#쿠킹박스', ...",2018-08-10,"2,825",NaN
633,-#점심 #자이언트미트볼파스타 #석류즙.간단하면서 맛있게 점심 먹기!!재료에 소스에...,"['#점심', '#자이언트미트볼파스타', '#석류즙.간단하면서', '#자이언트미트볼...",2018-08-09,99,NaN
634,🎀@powderroom.eventㆍ덥고 푹푹 찌는날 외식만 하다가 집에서도 맛있는 ...,"['#자이언트미트볼파스타', '#프레시지밀키트요리를', '#프레시지', '#밀키트'...",2018-08-06,94,NaN


In [69]:
fresheasymealkit.to_excel('../output/fresheasymealkit.xlsx')

### 2.1.4 word "프레시지쿠킹박스"

In [70]:
from selenium import webdriver
driver = webdriver.Chrome('../driver/chromedriver.exe')

import time
driver.get('https://www.instagram.com')
time.sleep(2)

email = '********'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '********'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)

input_pw.submit()
time.sleep(3)

In [73]:
word = '프레시지쿠킹박스'
url = insta_searching(word)

driver.get(url)
time.sleep(3)

# 첫 번째 게시글 열기
select_first(driver)

# 비어 있는 변수(result) 만들기
result = []

# 여러 게시물 수집하기
target = 500

for i in tqdm(range(target)):
    try:
        data = get_content(driver)
        result.append(data)
        move_next(driver)

    except:
        time.sleep(2)
        move_next(driver)

print(result[:2])


[['집밥피드올리기 냠냠🍴#메리프레시지 #크리스마스파티 도 문제없겠어요😘😘', ['#메리프레시지', '#크리스마스파티'], '2019-11-30', '150', ''], ['#독산동우리집 오늘은 엄마표갈비랑 #프레시지수제비떡볶이 꿀조합🍯♥️밀키트는 세상 간편한 프레시지가 왕짱인듯👍🏻.#집밥스타그램 #집밥 #집밥메뉴 #집밥그램 #집밥메뉴 #프레시지 #밀키트 #프레시지쿠킹박스 #프레시지밀키트 #프레시지떡볶이 #떡볶이 #떡볶이맛집 #독산동맛집 #금천구맛집 #맛스타그램 #먹스타그램 #먹스타맞팔 #먹스타소통 #일상 #데일리 #먹팔 #맞팔 #소통 #다이어트 #다이어트식단 #집밥은살안쪄', ['#독산동우리집', '#프레시지수제비떡볶이', '#집밥스타그램', '#집밥', '#집밥메뉴', '#집밥그램', '#집밥메뉴', '#프레시지', '#밀키트', '#프레시지쿠킹박스', '#프레시지밀키트', '#프레시지떡볶이', '#떡볶이', '#떡볶이맛집', '#독산동맛집', '#금천구맛집', '#맛스타그램', '#먹스타그램', '#먹스타맞팔', '#먹스타소통', '#일상', '#데일리', '#먹팔', '#맞팔', '#소통', '#다이어트', '#다이어트식단', '#집밥은살안쪄'], '2020-07-06', '284', '금천 롯데캐슬 골드파크']]


In [75]:
result_df = pd.DataFrame(result)
result_df.columns = ['content', 'tags', 'date', 'like', 'place']
result_df.to_excel('../output/fresheasycookingbox_1.xlsx')

driver.close()

In [76]:
word = '프레시지쿠킹박스'
url = insta_searching(word)

driver.get(url)
time.sleep(3)

# 첫 번째 게시글 열기
select_first(driver)

# 비어 있는 변수(result) 만들기
result = []

# 여러 게시물 수집하기
target = 500

for i in tqdm(range(target)):
    try:
        data = get_content(driver)
        result.append(data)
        move_next(driver)

    except:
        time.sleep(2)
        move_next(driver)

print(result[:2])
driver.close()


[['집밥피드올리기 냠냠🍴#메리프레시지 #크리스마스파티 도 문제없겠어요😘😘', ['#메리프레시지', '#크리스마스파티'], '2019-11-30', '150', ''], ['#독산동우리집 오늘은 엄마표갈비랑 #프레시지수제비떡볶이 꿀조합🍯♥️밀키트는 세상 간편한 프레시지가 왕짱인듯👍🏻.#집밥스타그램 #집밥 #집밥메뉴 #집밥그램 #집밥메뉴 #프레시지 #밀키트 #프레시지쿠킹박스 #프레시지밀키트 #프레시지떡볶이 #떡볶이 #떡볶이맛집 #독산동맛집 #금천구맛집 #맛스타그램 #먹스타그램 #먹스타맞팔 #먹스타소통 #일상 #데일리 #먹팔 #맞팔 #소통 #다이어트 #다이어트식단 #집밥은살안쪄', ['#독산동우리집', '#프레시지수제비떡볶이', '#집밥스타그램', '#집밥', '#집밥메뉴', '#집밥그램', '#집밥메뉴', '#프레시지', '#밀키트', '#프레시지쿠킹박스', '#프레시지밀키트', '#프레시지떡볶이', '#떡볶이', '#떡볶이맛집', '#독산동맛집', '#금천구맛집', '#맛스타그램', '#먹스타그램', '#먹스타맞팔', '#먹스타소통', '#일상', '#데일리', '#먹팔', '#맞팔', '#소통', '#다이어트', '#다이어트식단', '#집밥은살안쪄'], '2020-07-06', '284', '금천 롯데캐슬 골드파크']]


In [77]:
len(result)

981

In [78]:
result_df = pd.DataFrame(result)
result_df.columns = ['content', 'tags', 'date', 'like', 'place']
result_df.to_excel('../output/fresheasycookingbox.xlsx')

## 2.2 Naver Blog Crawling

### 2.2.1 word "프레시지"

In [ ]:
query_txt = '프레시지'
driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get('http://www.naver.com')
time.sleep(2)

In [ ]:
element = driver.find_element_by_id('query')
element.send_keys(query_txt)
element.submit()

driver.find_element_by_link_text('블로그').click()

In [ ]:
# 검색 옵션
driver.find_element_by_css_selector('#search_option_button').click()

#정렬 버튼 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click()

# 관련도순 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

result = []
for i in tqdm(range(1000)):

    #제목
    title = soup.select('a.api_txt_lines.total_tit')[i].text
    title = unicodedata.normalize('NFC', title)
    
    #본문
    content = soup.select('div.total_group')[i].text
    content = unicodedata.normalize('NFC', content)
    
    #날짜
    date = soup.select('span.sub_time.sub_txt')[i].text
    
    data = [title, content, date]
    result.append(data)

print(len(result))
driver.close()

In [195]:
results_df = pd.DataFrame(result)
results_df.columns = ['title','content','date']
results_df.drop_duplicates(subset=['content'], inplace=True)
results_df.reset_index(inplace = True, drop = True)

results_df.to_excel('../output/fresheasy_blog.xlsx')
results_df.tail()

,title,content,date
990,프레시지 감바스 알아히요 간단하고 맛있음!,지난 번 곱창전골이 땡겨서 프레시지를 처음 이용했을 때 곱창전골과 함께 구매한...,2020.09.17.
991,밀키트로 감바스만들기- 프레시지,#밀키트 #프레시지 #감바스만들기 #내배부르게먹기 #내돈내산 항상 아이들 맛있...,2020.10.18.
992,[간편조리추천] 프레시지 블랙라벨 스테이크 완전 추천!!...,바로바로바로바로바로바로 [프레시지 블랙라벨 스테이크] <협찬 절대절대 아님!!...,2020.07.06.
993,[자취요리_마라탕] 프레시지 사천마라탕/마라탕 만들기/B마트...,#마라탕만들기 #사천마라탕 #B마트마라탕 #프레시지마라탕 손쉽게 만드는 마라탕...,2020.09.03.
994,구매후기 : 프레시지 우삼겹 순두부찌개,프레시지 우삼겹 순두부찌개 몇달전에 드디어 집에서 독립해서 혼자하는 J’s 사...,2020.09.12.


### 2.2.2 word "프레시지밀키트"

In [ ]:
query_txt = '프레시지밀키트'
driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get('http://www.naver.com')
time.sleep(2)

element = driver.find_element_by_id('query')
element.send_keys(query_txt)
element.submit()

driver.find_element_by_link_text('블로그').click()

In [ ]:
# 검색 옵션
driver.find_element_by_css_selector('#search_option_button').click()

#정렬 버튼 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click()

# 관련도순 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

result = []
for i in tqdm(range(1000)):

    #제목
    title = soup.select('a.api_txt_lines.total_tit')[i].text
    title = unicodedata.normalize('NFC', title)
    
    #본문
    content = soup.select('div.total_group')[i].text
    content = unicodedata.normalize('NFC', content)
    
    #날짜
    date = soup.select('span.sub_time.sub_txt')[i].text
    
    data = [title, content, date]
    result.append(data)
    
driver.close()

In [ ]:
results_df = pd.DataFrame(result)
results_df.columns = ['title','content','date']
results_df.drop_duplicates(subset=['content'], inplace=True)
results_df.reset_index(inplace = True, drop = True)

print(len(results_df))
results_df.tail()

In [ ]:
results_df.to_excel('../output/fresheasymealkit_blog.xlsx')

### 2.2.3 word "프레시지쿠킹박스"

In [ ]:
query_txt = '프레시지쿠킹박스'
driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get('http://www.naver.com')
time.sleep(2)

element = driver.find_element_by_id('query')
element.send_keys(query_txt)
element.submit()

driver.find_element_by_link_text('블로그').click()

In [ ]:
# 검색 옵션
driver.find_element_by_css_selector('#search_option_button').click()

#정렬 버튼 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click()

# 관련도순 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

result = []
for i in tqdm(range(1000)):

    #제목
    title = soup.select('a.api_txt_lines.total_tit')[i].text
    title = unicodedata.normalize('NFC', title)
    
    #본문
    content = soup.select('div.total_group')[i].text
    content = unicodedata.normalize('NFC', content)
    
    #날짜
    date = soup.select('span.sub_time.sub_txt')[i].text
    
    data = [title, content, date]
    result.append(data)

In [ ]:
results_df = pd.DataFrame(result)
results_df.columns = ['title','content','date']
results_df.drop_duplicates(subset=['content'], inplace=True)
results_df.reset_index(inplace = True, drop = True)

print(results_df.tail())
len(results_df)
results_df.to_excel('../output/fresheasycookingbox_blog.xlsx')

## 2.3 Naver Cafe Crawling

### 2.3.1 word "프레시지"

In [ ]:
query_txt = '프레시지'

driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get('http://www.naver.com')
time.sleep(2)

element = driver.find_element_by_id('query')
element.send_keys(query_txt)
element.submit()

driver.find_element_by_link_text('카페').click()

In [ ]:
# 검색 옵션
driver.find_element_by_css_selector('#search_option_button').click()

#정렬 버튼 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click()

# 관련도순 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

result = []
for i in tqdm(range(1000)):

    #제목
    title = soup.select('a.api_txt_lines.total_tit')[i].text
    title = unicodedata.normalize('NFC', title)
    
    #본문
    content = soup.select('div.total_group')[i].text
    content = unicodedata.normalize('NFC', content)
    
    #날짜
    date = soup.select('span.sub_time.sub_txt')[i].text
    
    data = [title, content, date]
    result.append(data)

In [ ]:
results_df = pd.DataFrame(result)
results_df.columns = ['title','content','date']
results_df.drop_duplicates(subset=['content'], inplace=True)
results_df.reset_index(inplace = True, drop = True)
print(results_df.tail())
results_df.to_excel('../output/fresheasy/fresheasy_cafe.xlsx')
driver.close()

### 2.3.2 word "프레시지밀키트"

In [ ]:
query_txt = '프레시지밀키트'

driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get('http://www.naver.com')
time.sleep(2)

element = driver.find_element_by_id('query')
element.send_keys(query_txt)
element.submit()

driver.find_element_by_link_text('카페').click()

In [ ]:
# 검색 옵션
driver.find_element_by_css_selector('#search_option_button').click()

#정렬 버튼 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click()

# 관련도순 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

result = []
for i in tqdm(range(1000)):

    #제목
    title = soup.select('a.api_txt_lines.total_tit')[i].text
    title = unicodedata.normalize('NFC', title)
    
    #본문
    content = soup.select('div.total_group')[i].text
    content = unicodedata.normalize('NFC', content)
    
    #날짜
    date = soup.select('span.sub_time.sub_txt')[i].text
    
    data = [title, content, date]
    result.append(data)
    
print(len(result))

In [ ]:
results_df = pd.DataFrame(result)
results_df.columns = ['title','content','date']

results_df.drop_duplicates(subset=['content'], inplace=True)
results_df.reset_index(inplace = True, drop = True)
results_df.to_excel('../output/fresheasy/fresheasymealkit_cafe.xlsx')

driver.close()

### 2.3.3 word "프레시지쿠킹박스"

In [ ]:
query_txt = '프레시지쿠킹박스'

driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get('http://www.naver.com')
time.sleep(2)

element = driver.find_element_by_id('query')
element.send_keys(query_txt)
element.submit()

driver.find_element_by_link_text('카페').click()

In [ ]:
# 검색 옵션
driver.find_element_by_css_selector('#search_option_button').click()

#정렬 버튼 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click()

# 관련도순 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

result = []
for i in tqdm(range(1000)):

    #제목
    title = soup.select('a.api_txt_lines.total_tit')[i].text
    title = unicodedata.normalize('NFC', title)
    
    #본문
    content = soup.select('div.total_group')[i].text
    content = unicodedata.normalize('NFC', content)
    
    #날짜
    date = soup.select('span.sub_time.sub_txt')[i].text
    
    data = [title, content, date]
    result.append(data)
len(result)

In [ ]:
results_df = pd.DataFrame(result)
results_df.columns = ['title','content','date']

results_df.drop_duplicates(subset=['content'], inplace=True)
results_df.reset_index(inplace = True, drop = True)

results_df.to_excel('../output/fresheasy/fresheasycookingbox_cafe.xlsx')
driver.close()

### 2.3.4 word "프레시지+쓰레기"

#### Blog Crawling

In [ ]:
query_txt = '프레시지+쓰레기'

driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get('http://www.naver.com')
time.sleep(2)

element = driver.find_element_by_id('query')
element.send_keys(query_txt)
element.submit()

driver.find_element_by_link_text('블로그').click()

In [ ]:
# 검색 옵션
driver.find_element_by_css_selector('#search_option_button').click()

#정렬 버튼 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click()

# 관련도순 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

result = []
for i in tqdm(range(762)):

    #제목
    title = soup.select('a.api_txt_lines.total_tit')[i].text
    title = unicodedata.normalize('NFC', title)
    
    #본문
    content = soup.select('div.total_group')[i].text
    content = unicodedata.normalize('NFC', content)
    
    #날짜
    date = soup.select('span.sub_time.sub_txt')[i].text
    
    data = [title, content, date]
    result.append(data)
len(result)

In [ ]:
results_df = pd.DataFrame(result)
results_df.columns = ['title','content','date']

results_df.drop_duplicates(subset=['content'], inplace=True)
results_df.reset_index(inplace = True, drop = True)
results_df.to_excel('../output/fresheasy/fresheasy_trash_blog.xlsx')
driver.close()

#### Cafe Crawling

In [ ]:
driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get('http://www.naver.com')
time.sleep(2)

element = driver.find_element_by_id('query')
element.send_keys(query_txt)
element.submit()

driver.find_element_by_link_text('카페').click()

In [ ]:
# 검색 옵션
driver.find_element_by_css_selector('#search_option_button').click()

#정렬 버튼 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click()

# 관련도순 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

result = []
for i in tqdm(range(80)):

    #제목
    title = soup.select('a.api_txt_lines.total_tit')[i].text
    title = unicodedata.normalize('NFC', title)
    
    #본문
    content = soup.select('div.total_group')[i].text
    content = unicodedata.normalize('NFC', content)
    
    #날짜
    date = soup.select('span.sub_time.sub_txt')[i].text
    
    data = [title, content, date]
    result.append(data)

In [ ]:
results_df = pd.DataFrame(result)
results_df.columns = ['title','content','date']

results_df.drop_duplicates(subset=['content'], inplace=True)
results_df.reset_index(inplace = True, drop = True)
results_df['title']

In [ ]:
results_df.to_excel('../output/fresheasy/fresheasy_trash_cafe.xlsx')
driver.close()

#### 크롤링한 데이터 합치기

In [ ]:
trash_blog = pd.read_excel('../output/fresheasy/fresheasy_trash_blog.xlsx',
                          engine = 'openpyxl')
trash_cafe = pd.read_excel('../output/fresheasy/fresheasy_trash_cafe.xlsx',
                          engine = 'openpyxl')

In [ ]:
trash_total = pd.concat([trash_blog, trash_cafe])
trash_total.drop_duplicates(subset=['content'], inplace=True)
trash_total.reset_index(inplace=True, drop=True)
trash_total.to_excel('../output/fresheasy/fresheasy_trash_naver.xlsx',index=False)

### 2.3.5 word "프레시지+포장"

#### Blog Crawling

In [ ]:
query_txt = '프레시지+포장'
driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get('http://www.naver.com')
time.sleep(2)

element = driver.find_element_by_id('query')
element.send_keys(query_txt)
element.submit()

driver.find_element_by_link_text('블로그').click()

In [ ]:
# 검색 옵션
driver.find_element_by_css_selector('#search_option_button').click()

#정렬 버튼 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click()

# 관련도순 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

result = []
for i in tqdm(range(1000)):

    #제목
    title = soup.select('a.api_txt_lines.total_tit')[i].text
    title = unicodedata.normalize('NFC', title)
    
    #본문
    content = soup.select('div.total_group')[i].text
    content = unicodedata.normalize('NFC', content)
    
    #날짜
    date = soup.select('span.sub_time.sub_txt')[i].text
    
    data = [title, content, date]
    result.append(data)

In [ ]:
results_df = pd.DataFrame(result)
results_df.columns = ['title','content','date']
print(len(results_df))

results_df.drop_duplicates(subset=['content'], inplace=True)
results_df.reset_index(inplace = True, drop = True)
results_df.to_excel('../output#### Cafe Crawling/fresheasy/fresheasy_pack_blog.xlsx',index=False)
driver.close()

#### Cafe Crawling

In [ ]:
driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get('http://www.naver.com')
time.sleep(2)

element = driver.find_element_by_id('query')
element.send_keys(query_txt)
element.submit()

driver.find_element_by_link_text('카페').click()

In [ ]:
# 검색 옵션
driver.find_element_by_css_selector('#search_option_button').click()

#정렬 버튼 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click()

# 관련도순 xpath
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

result = []
for i in tqdm(range(1000)):

    #제목
    title = soup.select('a.api_txt_lines.total_tit')[i].text
    title = unicodedata.normalize('NFC', title)
    
    #본문
    content = soup.select('div.total_group')[i].text
    content = unicodedata.normalize('NFC', content)
    
    #날짜
    date = soup.select('span.sub_time.sub_txt')[i].text
    
    data = [title, content, date]
    result.append(data)

In [ ]:
results_df = pd.DataFrame(result)
results_df.columns = ['title','content','date']
print(len(results_df))

results_df.drop_duplicates(subset=['content'], inplace=True)
results_df.reset_index(inplace = True, drop = True)
results_df.to_excel('../output/fresheasy/fresheasy_pack_cafe.xlsx',index=False)

### 2.4 크롤링한 데이터 병합

#### 2.4.1 Naver Blog

In [ ]:
fresheasy = pd.read_excel('../output/fresheasy/fresheasy_blog.xlsx', engine='openpyxl', index_col = 0)

fresheasymealkit = pd.read_excel('../output/fresheasy/fresheasymealkit_blog.xlsx',
                                engine = 'openpyxl', index_col=0)

fresheasycookingbox = pd.read_excel('../output/fresheasy/fresheasycookingbox_blog.xlsx',
                                   engine = 'openpyxl', index_col = 0)

In [ ]:
fresheasy_total_blog.drop_duplicates(subset=['content'], inplace=True)
fresheasy_total_blog.reset_index(inplace=True, drop=True)
len(fresheasy_total_blog)
fresheasy_total_blog = pd.concat([fresheasy, fresheasymealkit, fresheasycookingbox])
fresheasy_total_blog.to_excel('../output/fresheasy/fresheasy_total_blog.xlsx', index=False)

#### 2.4.2 Naver Cafe

In [ ]:
fresheasy1 = pd.read_excel('../output/fresheasy/fresheasy_cafe.xlsx', 
                           engine='openpyxl', index_col = 0)

fresheasymealkit1 = pd.read_excel('../output/fresheasy/fresheasymealkit_cafe.xlsx',
                                engine = 'openpyxl', index_col=0)

fresheasycookingbox1 = pd.read_excel('../output/fresheasy/fresheasycookingbox_cafe.xlsx',
                                   engine = 'openpyxl', index_col = 0)

In [ ]:
fresheasy_total_cafe = pd.concat([fresheasy1, fresheasymealkit1, fresheasycookingbox1])
fresheasy_total_cafe.drop_duplicates(subset=['content'], inplace=True)
fresheasy_total_cafe.reset_index(inplace=True, drop=True)
len(fresheasy_total_cafe)

In [ ]:
fresheasy_total_cafe = pd.read_excel('../output/fresheasy/fresheasy_total_cafe.xlsx',
                                    engine = 'openpyxl')

#### 2.4.3 Instagram

In [ ]:
fresheasy2 = pd.read_excel('../output/fresheasy/fresheasy_insta.xlsx', engine='openpyxl', index_col = 0)

fresheasymealkit2 = pd.read_excel('../output/fresheasy/fresheasymealkit_insta.xlsx',
                                engine = 'openpyxl', index_col=0)

fresheasycookingbox2 = pd.read_excel('../output/fresheasy/fresheasycookingbox_insta.xlsx',
                                   engine = 'openpyxl', index_col = 0)

In [ ]:
fresheasy_total_insta = pd.concat([fresheasy2, fresheasymealkit2, fresheasycookingbox2])
fresheasy_total_insta.drop_duplicates(subset=['content'], inplace=True)
fresheasy_total_insta.reset_index(inplace=True, drop=True)
len(fresheasy_total_insta)

In [ ]:
fresheasy_total_insta.to_excel('../output/fresheasy/fresheasy_total_insta.xlsx', index=False)